#Agrupamiento de Semillas

El grupo examinado estuvo compuesto por granos pertenecientes a tres variedades diferentes de trigo: Kama, Rosa y Canadian, de 70 elementos cada uno, seleccionados al azar para
el experimento. Se detectó una visualización de alta calidad de la estructura interna del núcleo utilizando una técnica de rayos X suaves. No es destructivo y es considerablemente más económico que otras técnicas de imagen más sofisticadas, como la microscopía de barrido o la tecnología láser. Las imágenes se registraron en placas KODAK de rayos X de 13x18 cm. Los estudios se llevaron a cabo utilizando granos de trigo cosechados con cosechadora provenientes de campos experimentales, explorados en el Instituto de Agrofísica de la Academia de Ciencias de Polonia en Lublin.

El conjunto de datos se puede utilizar para las tareas de clasificación y análisis de conglomerados.


Información de atributos:

Para construir los datos, se midieron siete parámetros geométricos de los granos de trigo:

1. área A,
2. perímetro P,
3. compacidad C = 4*pi*A/P^2,
4. longitud del grano,
5. ancho del grano,
6. coeficiente de asimetría
7. longitud del surco del grano.

Todos estos parámetros fueron continuos de valor real.

¡Intentaremos clasificarlos en 3 grupos!

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster02').getOrCreate()

In [0]:
#cargar datos
ruta = 'dbfs:/FileStore/shared_uploads/jgamarramoreno@gmail.com/seeds_dataset.csv'
datos = spark.read.csv(ruta,header=True,inferSchema=True)

In [0]:
datos.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355|             5.175|
|14.38|    14.21|     0.8951|             5.386|             3.312|   2.4619999999999997|             4.956|
|14.69|    14.49|  

In [0]:
datos.head()

Out[6]: Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)

In [0]:
datos.describe().show()

+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|summary|              area|         perimeter|         compactness|   length_of_kernel|   width_of_kernel|asymmetry_coefficient|   length_of_groove|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|  count|               210|               210|                 210|                210|               210|                  210|                210|
|   mean|14.847523809523816|14.559285714285718|  0.8709985714285714|  5.628533333333335| 3.258604761904762|   3.7001999999999997|  5.408071428571429|
| stddev|2.9096994306873647|1.3059587265640225|0.023629416583846364|0.44306347772644983|0.3777144449065867|   1.5035589702547392|0.49148049910240543|
|    min|             10.59|             12.41|              0.8081|              4.899|            

### Formateo de datos

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
datos.columns

Out[12]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [0]:
vec_ensamblador = VectorAssembler(inputCols=datos.columns,
                                  outputCol='caracteristicas')

In [0]:
datos_final = vec_ensamblador.transform(datos)

In [0]:
datos_final.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|     caracteristicas|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|[15.26,14.84,0.87...|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|[14.88,14.57,0.88...|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|[14.29,14.09,0.90...|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|[13.84,13.94,0.89...|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355| 

### Estandarizar los datos

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
escala = StandardScaler(inputCol='caracteristicas',
                        outputCol='caracteristicasEscala',
                        withMean=False,
                        withStd=True)

In [0]:
modeloEscala = escala.fit(datos_final)

In [0]:
datos_finales_escala = modeloEscala.transform(datos_final)

In [0]:
datos_finales_escala.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+---------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|     caracteristicas|caracteristicasEscala|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+---------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|[15.26,14.84,0.87...| [5.24452795332028...|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|[14.88,14.57,0.88...| [5.11393027165175...|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|[14.29,14.09,0.90...| [4.91116018695588...|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259

### Crear el modelo y entrenarlo

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
kmeans = KMeans(featuresCol='caracteristicasEscala',k=3)

In [0]:
modelo = kmeans.fit(datos_finales_escala)

Mostrar los resultados

In [0]:
centroides = modelo.clusterCenters()
print("Centroides de Clusters:")
for centroide in centroides:
    print(centroide)

Centroides de Clusters:
[ 4.93382436 10.94691274 37.30542404 12.41332714  8.60366812  1.82917353
 10.40106154]
[ 6.35645488 12.40730852 37.41990178 13.93860446  9.7892399   2.41585013
 12.29286107]
[ 4.06660859 10.14191893 35.84098009 11.81592066  7.52397236  3.1823335
 10.39801233]


Predicciones

In [0]:
modelo.transform(datos_finales_escala).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows

